In [ ]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd , numpy as np
import plotly.express as px
!pip install unidecode


# CSV dosyasının yolu
dosya_yolu = '/content/drive/MyDrive/CSV/FM 2023.csv'

# CSV dosyasını oku
df = pd.read_csv(dosya_yolu)
fm=df.copy()

Gerçek veri setimizin üstünde işlem yapmamak için başka bir yere kopyaladık:

In [ ]:
headers={
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"

}


In [ ]:
from unidecode import unidecode

def Convert_to_latin(string):
    return unidecode(string)


In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from unidecode import unidecode

def Convert_to_latin(text):
    return unidecode(text)

def Find_Player_Url(player_name):
    search_url = f"https://www.transfermarkt.com/schnellsuche/ergebnis/schnellsuche?query={player_name}&x=0&y=0"

    # Headers tanımlama
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    try:
        response = requests.get(search_url, headers=headers)

        if response.status_code == 200:
            html_content = response.content
            soup = BeautifulSoup(html_content, 'html.parser')

            # Oyuncu adına göre ilk sonuçtan URL'yi çıkartma
            player_link = soup.find('a', {'title': player_name})

            if player_link:
                player_url = player_link['href']
                full_url = f"https://www.transfermarkt.com{player_url}"
                return full_url
            else:
                # Oyuncu bağlantısı bulunamadı, Latin alfabesine çevrilmiş oyuncu adı ile tekrar dene
                latin_player_name = Convert_to_latin(player_name)
                if latin_player_name != player_name:
                    return Find_Player_Url(latin_player_name)
                else:
                    print("Oyuncu bağlantısı bulunamadı.")
        else:
            print(f"HTTP Hatası: {response.status_code}")
    except Exception as e:
        print(f"Hata oluştu: {e}")



In [ ]:

def Find_Player_UrlDetail(player_name):
    search_url = f"https://www.transfermarkt.com/schnellsuche/ergebnis/schnellsuche?query={player_name}&x=0&y=0"

    # Headers tanımlama
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    try:
        response = requests.get(search_url, headers=headers)

        if response.status_code == 200:
            html_content = response.content
            soup = BeautifulSoup(html_content, 'html.parser')

            # Oyuncu adına göre ilk sonuçtan URL'yi çıkartma
            #player_link = soup.find('a', {'class': 'spielprofil_tooltip', 'title': player_name})
            player_link = soup.find('a', {'title': player_name, 'href': re.compile(r'/profil/spieler/\d+')})



            if player_link:

                player_url = player_link['href']
                full_url = f"https://www.transfermarkt.com{player_url}"

                return full_url
            else:
                # Arama sonuçlarını kontrol et
                search_results = soup.find('div', {'class': 'search-results'})
                print(search_results)
        else:
            print(f"HTTP Hatası: {response.status_code}")
    except Exception as e:
        basic_url=Find_Player_Url(player_name)
        print(f"Hata oluştu: {e}")
        return basic_url

# Örnek kullanım
player_name = "Mehdi Ghaedi"
url = Find_Player_UrlDetail(player_name)
print(f"{player_name}'nin Transfermarkt profili: {url}")


None
Mehdi Ghaedi'nin Transfermarkt profili: None


In [ ]:
def Get_Market_Value(player_name):

    url = Find_Player_Url(player_name)

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    market_value_element = soup.find('a', class_='data-header__market-value-wrapper')

    if market_value_element:
        market_value_text = market_value_element.text.strip()
        numeric_part = re.search(r'\d+\.\d+|\d+', market_value_text)
        market_value = float(numeric_part.group()) if numeric_part else None
        return market_value
    else:
        return 0


market_value_result = Get_Market_Value("Neymar")

if market_value_result is not None:
    print("Market Value:", market_value_result)
else:
    print("Market value element not found.")

Market Value: 45.0


In [ ]:
fm['Values'] = fm['Values'].astype(float)
fm.info()

In [ ]:

market_value_resul = Get_Market_Value(fm.iloc[0]["Name"])
print(market_value_resul)


60.0


In [ ]:
hatasayacı=0

In [ ]:
for satir_indeksi, satir in df.iterrows():

    try:

        # Her satır için bir işlem yapalım (örneğin, Name ve Values sütunlarını yazdıralım)
        sutun_adi = 'Values'
        istenen_deger = Get_Market_Value(fm.iloc[satir_indeksi]["Name"])  # Değiştirmek istediğiniz yeni değer
        if(istenen_deger>181):
          istenen_deger=istenen_deger/1000


        fm.at[satir_indeksi, sutun_adi] = istenen_deger
        print(str(fm.iloc[satir_indeksi]["Name"]) +" " +str(fm.iloc[satir_indeksi]["Values"]))


        fm.to_csv('/content/drive/MyDrive/CSV/fmValues.csv', index=False)


    except Exception as e:
        print("Hata:", e)
        print("Hata Alınan index değeri"+str(satir_indeksi))
        print(hatasayacı)
        hatasayacı+=1
        continue  # Hata alındığında döngüyü sonraki satıra geçerek devam ettir

print("Döngü tamamlandı.")

In [ ]:
dosya_yolu = '/content/drive/MyDrive/CSV/fmValues.csv'

# CSV dosyasını oku
a= pd.read_csv(dosya_yolu)
a.info()

In [ ]:
buyuk_oyuncular = fm[fm['Values'] > 181]

print(buyuk_oyuncular)

In [ ]:
for satir_indeksi, satir in buyuk_oyuncular.iterrows():

    try:

        # Her satır için bir işlem yapalım (örneğin, Name ve Values sütunlarını yazdıralım)
        sutun_adi = 'Values'
        istenen_deger = Get_Market_Value(fm.iloc[satir_indeksi]["Name"])  # Değiştirmek istediğiniz yeni değer
        if(istenen_deger>181):
          istenen_deger=istenen_deger/1000


        fm.at[satir_indeksi, sutun_adi] = istenen_deger
        print(str(fm.iloc[satir_indeksi]["Name"]) +" " +str(fm.iloc[satir_indeksi]["Values"]))


        fm.to_csv('/content/drive/MyDrive/CSV/fmValues.csv', index=False)


    except Exception as e:
        print("Hata:", e)
        print("Hata Alınan index değeri"+str(satir_indeksi))
        print(hatasayacı)
        hatasayacı+=1
        continue  # Hata alındığında döngüyü sonraki satıra geçerek devam ettir

print("Döngü tamamlandı.")